In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os

print(tf.__version__)

1.14.0


In [89]:
import pandas as pd
df = pd.read_json("moviedata2.json")


In [90]:
import re

In [91]:
df.head()

movie_genre  \
0  [Adventure, Drama, Horror, Thriller]   
1              [Drama, Fantasy, Horror]   
2                      [Horror, Sci-Fi]   
3           [Horror, Mystery, Thriller]   
4           [Adventure, Drama, Fantasy]   

                                          movie_name  \
0                           47 Meters Down: Uncaged혻   
1                                    It Chapter Two혻   
2                                             Alien혻   
3                                   The Conjuring 3혻   
4  The Lord of the Rings: The Fellowship of the R...   

                                        movie_poster  \
0  www.imdb.com/title/tt7329656/mediaviewer/rm242...   
1  www.imdb.com/title/tt7349950/mediaviewer/rm324...   
2  www.imdb.com/title/tt0078748/mediaviewer/rm299...   
3  www.imdb.com/title/tt7069210/mediaviewer/rm175...   
4  www.imdb.com/title/tt0120737/mediaviewer/rm359...   

                                       movie_summary  \
0  [Four teen girls diving in a ruined underwater...   
1  [Twenty-seven years after their first encounte...   
2  [After a space merchant vessel perceives an un...   
3                                                 []   
4  [A meek Hobbit from the Shire and eight compan...   

                                      movie_synopsis  \
0  [The film starts at a girls school in Yucatan,...   
1  [In 2016 Derry, Maine, a young gay man is beat...   
2  [The opening credits appear in front of a larg...   
3                                                 []   
4  [The prologue, spoken by Galadriel, shows the ...   

                                       movie_tagline  
0                 ['\n', '\nThe Next Chapter', '\n']  
1               ['\n', "\nYou'll Float Again", '\n']  
2   ['\n', "\nIt's Alien, the 8th passenger.", '\n']  
3     ['\n', '\n', '\xa0', '\n', '\xa0', '\n', '\n']  
4  ['\n', '\nAll we have to decide is what to do ...

In [97]:
df.movie_genre = df.movie_genre.apply(lambda x: str(x))
df.head()

movie_genre  \
0  ['Adventure', 'Drama', 'Horror', 'Thriller']   
1                ['Drama', 'Fantasy', 'Horror']   
2                          ['Horror', 'Sci-Fi']   
3             ['Horror', 'Mystery', 'Thriller']   
4             ['Adventure', 'Drama', 'Fantasy']   

                                          movie_name  \
0                           47 Meters Down: Uncaged혻   
1                                    It Chapter Two혻   
2                                             Alien혻   
3                                   The Conjuring 3혻   
4  The Lord of the Rings: The Fellowship of the R...   

                                        movie_poster  \
0  www.imdb.com/title/tt7329656/mediaviewer/rm242...   
1  www.imdb.com/title/tt7349950/mediaviewer/rm324...   
2  www.imdb.com/title/tt0078748/mediaviewer/rm299...   
3  www.imdb.com/title/tt7069210/mediaviewer/rm175...   
4  www.imdb.com/title/tt0120737/mediaviewer/rm359...   

                                       movie_summary  \
0  [Four teen girls diving in a ruined underwater...   
1  [Twenty-seven years after their first encounte...   
2  [After a space merchant vessel perceives an un...   
3                                                 []   
4  [A meek Hobbit from the Shire and eight compan...   

                                      movie_synopsis  \
0  [The film starts at a girls school in Yucatan,...   
1  [In 2016 Derry, Maine, a young gay man is beat...   
2  [The opening credits appear in front of a larg...   
3                                                 []   
4  [The prologue, spoken by Galadriel, shows the ...   

                                       movie_tagline  
0                                   The Next Chapter  
1                                 You'll Float Again  
2                     It's Alien, the 8th passenger.  
3                                                     
4  All we have to decide is what to do with the t...

In [98]:
df.movie_genre = df.movie_genre.apply([", '".join([j for j in eval(i) if ])"'])

In [99]:
df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

In [100]:
df.movie_summary = df.movie_summary.apply(lambda x: str(x))
df.movie_summary = df.movie_summary.apply(lambda x: x.replace('\[|"|\]','').replace("]|'",''))
df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x))
df.movie_synopsis = df.movie_synopsis.apply(lambda x : x.replace('\[|"|\]','').replace(']',''))
df.movie_tagline = df.movie_tagline.apply(lambda x: str(x))




In [92]:
tl = list(df["movie_tagline"])
tl = [" ".join([ j for j in eval(i) if j != "\n"]).strip() for i in tl]

In [96]:
df.movie_tagline = tl
df

movie_genre  \
0             [Adventure, Drama, Horror, Thriller]   
1                         [Drama, Fantasy, Horror]   
2                                 [Horror, Sci-Fi]   
3                      [Horror, Mystery, Thriller]   
4                      [Adventure, Drama, Fantasy]   
5                         [Crime, Drama, Thriller]   
6                     [Adventure, Family, Fantasy]   
7                        [Comedy, Family, Fantasy]   
8             [Action, Adventure, Comedy, Fantasy]   
9           [Animation, Adventure, Comedy, Family]   
10                          [Action, Drama, Sport]   
11                   [Action, Adventure, Thriller]   
12                     [Action, Adventure, Sci-Fi]   
13     [Action, Adventure, Crime, Drama, Thriller]   
14                      [Crime, Mystery, Thriller]   
15                       [Drama, Horror, Thriller]   
16                              [Biography, Drama]   
17                      [Drama, Mystery, Thriller]   
18                     [Horror, Mystery, Thriller]   
19                     [Action, Adventure, Sci-Fi]   
20                                         [Drama]   
21                         [Action, Crime, Horror]   
22                       [Action, Comedy, Mystery]   
23                                [Drama, Western]   
24                                  [Drama, Sport]   
25                   [Action, Adventure, Thriller]   
26                    [Action, Adventure, Fantasy]   
27                     [Action, Adventure, Sci-Fi]   
28             [Action, Adventure, Comedy, Sci-Fi]   
29                                         [Drama]   
...                                            ...   
43936                                [Documentary]   
43937                                     [Comedy]   
43938                             [Drama, History]   
43939                                      [Drama]   
43940      [Animation, Adventure, Family, Fantasy]   
43941                      [Comedy, Drama, Family]   
43942              [Crime, Drama, Music, Thriller]   
43943                                           []   
43944                                      [Drama]   
43945           [Action, Drama, Romance, Thriller]   
43946                             [Music, Romance]   
43947             [Action, Crime, Drama, Thriller]   
43948                                [Documentary]   
43949                                    [Romance]   
43950                              [Action, Drama]   
43951                  [Adventure, Comedy, Horror]   
43952                                     [Comedy]   
43953                         [Documentary, Drama]   
43954                     [Documentary, Adventure]   
43955                                      [Drama]   
43956                                  [Animation]   
43957                                     [Horror]   
43958                                     [Horror]   
43959                            [Comedy, Romance]   
43960                               [Crime, Drama]   
43961                    [Crime, Drama, Film-Noir]   
43962                                      [Drama]   
43963                        [Crime, Drama, Music]   
43964                                      [Drama]   
43965                                      [Drama]   

                                              movie_name  \
0                               47 Meters Down: Uncaged혻   
1                                        It Chapter Two혻   
2                                                 Alien혻   
3                                       The Conjuring 3혻   
4      The Lord of the Rings: The Fellowship of the R...   
5                                              The Mule혻   
6                 Harry Potter and the Sorcerer's Stone혻   
7                                           Hocus Pocus혻   
8                        Jumanji: Welcome to the Jungle혻   
9                                          Finding Nemo혻   
10                            

In [104]:
summ = list(df["movie_summary"])
summ

['["Four teen girls diving in a ruined underwater city quickly learn they\'ve entered the territory of the deadliest shark species in the claustrophobic labyrinth of submerged caves.", \'47 Meters Down: Uncaged follows the diving adventure of four teenage girls (Sophie N챕lisse, Corinne Foxx, Brianne Tju and Sistine Stallone) exploring a submerged Mayan City. Once inside, their rush of excitement turns into a jolt of terror as they discover the sunken ruins are a hunting ground for deadly Great White Sharks. With their air supply steadily dwindling, the friends must navigate the underwater labyrinth of claustrophobic caves and eerie tunnels in search of a way out of their watery hell.\']',
 '[\'Twenty-seven years after their first encounter with the terrifying Pennywise, the Losers Club have grown up and moved away, until a devastating phone call brings them back.\', "Defeated by members of the Losers\' Club, the evil clown Pennywise returns 27 years later to terrorize the town of Derry

In [2]:
sources = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
targets = [['나는', '배가', '고프다'],
           ['텐서플로우는', '매우', '어렵다'],
           ['텐서플로우는', '딥러닝을', '위한', '프레임워크이다'],
           ['텐서플로우는', '매우', '빠르게', '변화한다']]

In [3]:
# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}

pprint(source2idx)

{'<pad>': 0,
 'I': 1,
 'a': 2,
 'changing': 3,
 'deep': 4,
 'difficult': 5,
 'fast': 6,
 'feel': 7,
 'for': 8,
 'framework': 9,
 'hungry': 10,
 'is': 11,
 'learning': 12,
 'tensorflow': 13,
 'very': 14}
